In [1]:
import os
from sklearn.model_selection import train_test_split
from shutil import move
from ultralytics import YOLO
import cv2
from PIL import Image
import glob
import albumentations as A
import numpy as np
from shutil import copy

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.6 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [11]:


class_mapping = {
    "Car": 0,
    "Van": 1,
    "Truck": 2,
    "Pedestrian": 3,
    "Person_sitting": 4,
    "Cyclist": 5,
    "Tram": 6,
    "Misc": 7,
    "DontCare": -1  # Typically ignored in training
}


def convert_kitti_to_yolo_folder(kitti_label_folder, output_folder, img_width, img_height):
    for filename in os.listdir(kitti_label_folder):
        if filename.endswith(".txt"):
            kitti_label_file = os.path.join(kitti_label_folder, filename)
            with open(kitti_label_file, "r") as file:
                lines = file.readlines()

            yolo_labels = []
            for line in lines:
                parts = line.strip().split()
                class_name = parts[0]
                class_id = class_mapping[class_name]
                if class_name == "DontCare":
                    continue

                # Parse bounding box
                x_min = float(parts[4])
                y_min = float(parts[5])
                x_max = float(parts[6])
                y_max = float(parts[7])

                x_center = (x_min + x_max) / 2 / img_width
                y_center = (y_min + y_max) / 2 / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height

                
                yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

            os.makedirs(output_folder, exist_ok=True)
            output_file_path = os.path.join(output_folder, filename)
            with open(output_file_path, "w") as out_file:
                out_file.write("\n".join(yolo_labels))

# Usage:
convert_kitti_to_yolo_folder("annotations", "training/labels", 1242, 375)

In [12]:
def count_files(folder_path):
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return 0
    
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not files:
        print(f"Folder '{folder_path}' is empty.")
        return 0

    return len(files)



img_count = count_files('val/images')
label_count = count_files('val/images')

if img_count<=1495 or label_count<=1495:
    images_folder = 'training/images'
    annotations_folder = "training/labels"
    
    #os.makedirs(train_folder, exist_ok=True)
    
    for filename in os.listdir(images_folder):
        src_path = os.path.join(images_folder, filename)  # Construct full file path
        #dst_path = os.path.join(train_folder, filename)
        # copy2(src_path, dst_path)
    
    # Function to get the list of all images and annotations
    def get_files(folder_path):
        return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # Get the list of images and annotations
    image_files = get_files(images_folder)
    annotation_files = get_files(annotations_folder)
    
    # Split data into training and validation sets
    train_images, val_images, train_annotations, val_annotations = train_test_split(
        image_files, annotation_files, test_size=0.2, random_state=42
    )
    
    # Paths for saving validation data
    validation_images_folder = "val/images"
    validation_annotations_folder = "val/labels"
    
    os.makedirs(validation_images_folder, exist_ok=True)
    os.makedirs(validation_annotations_folder, exist_ok=True)
    
    # Function to copy files into validation folders
    
            
    def move_to_validation(files, destination_folder):
        for file_path in files:
            move(file_path, destination_folder)
    if img_count<=1495: # can be removed if you start from scratch
        move_to_validation(val_images, validation_images_folder)
    if label_count<=1495: # can be removed if you start from scratch
        move_to_validation(val_annotations, validation_annotations_folder)
    
    print("Data has been successfully split!")

Folder 'val/images' is empty.
Folder 'val/images' is empty.
Data has been successfully split!


In [38]:
def count_classes(labels_dir):
    class_counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
    
    for label_file in glob.glob(os.path.join(labels_dir, "*.txt")):
        with open(label_file, 'r') as f:
            for line in f:
                class_id = int(float(line.strip().split()[0]))
                class_counts[class_id] += 1
    
    return class_counts

# Check train dataset
train_counts = count_classes("training/labels")
aug=count_classes("training/augmented_labels")
print("Training set class distribution:", train_counts,aug)
print()
# Check validation dataset
val_counts = count_classes("val/labels")
print("Validation set class distribution:", val_counts)

{0: 92783, 1: 13516, 2: 7943, 3: 12882, 4: 2384, 5: 8007, 6: 7782, 7: 6738}
Validation set class distribution: {0: 5889, 1: 570, 2: 227, 3: 916, 4: 39, 5: 308, 6: 95, 7: 187}


In [25]:
# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.RandomScale(scale_limit=0.1, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5),
],
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels'])
)

def augment_minority_classes(images_dir, labels_dir, output_images_dir, output_labels_dir, class_id, augmentation_factor):
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)
    
    # Collect all label files containing the minority class
    minority_files = []
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith('.txt'):
            continue
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, 'r') as f:
            for line in f:
                class_val = line.strip().split()[0]
                if int(float(class_val)) == class_id:
                    minority_files.append(label_file.replace('.txt', ''))
                    break
    
    # Augment each image containing the minority class
    for file_base in minority_files:
        img_path = os.path.join(images_dir, file_base + '.png').replace("\\", "/")
        label_path = os.path.join(labels_dir, file_base + '.txt').replace("\\", "/")
        
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Image {img_path} not found or unreadable.")
            continue
        height, width = img.shape[:2]
        
        # Read bounding boxes
        with open(label_path, 'r', encoding='utf-8') as f:
            labels = [line.strip() for line in f]
        
        bboxes = []
        class_ids = []
        for label in labels:
            parts = label.split()
            cls_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:5])
            bboxes.append([x_center, y_center, w, h])
            class_ids.append(cls_id)
        
        # Create multiple augmented versions
        for i in range(augmentation_factor):
            transformed = transform(image=img, bboxes=bboxes, labels=class_ids)
            aug_img = transformed['image']
            aug_bboxes = transformed['bboxes']
            aug_class_ids = transformed['labels']
            
            # Save augmented image
            aug_img_path = os.path.join(output_images_dir, f"{file_base}_aug{i}.jpg").replace("\\", "/")
            cv2.imwrite(aug_img_path, aug_img)
            
            # Save augmented labels
            aug_label_path = os.path.join(output_labels_dir, f"{file_base}_aug{i}.txt").replace("\\", "/")
            with open(aug_label_path, 'w') as f:
                for j in range(len(aug_bboxes)):
                    x_center, y_center, w_box, h_box = aug_bboxes[j]
                    cls = aug_class_ids[j]
                    
                    # Clip values to [0,1] to avoid bounding box errors
                    x_center = np.clip(x_center, 0, 1)
                    y_center = np.clip(y_center, 0, 1)
                    w_box = np.clip(w_box, 0, 1)
                    h_box = np.clip(h_box, 0, 1)
                    
                    f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w_box:.6f} {h_box:.6f}\n")

In [29]:
augmented_images_dir = "training/augmented_images"
augmented_labels_dir = "training/augmented_labels"


augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=1,  
    augmentation_factor=3
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=2,  
    augmentation_factor=8
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=3,
    augmentation_factor=2  # Convert to integer - will create 1 augmented image per original
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=4,
    augmentation_factor=30  # Convert to integer - will create 1 augmented image per original
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=5,
    augmentation_factor=5  # Convert to integer - will create 1 augmented image per original
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=6,
    augmentation_factor=20  # Convert to integer - will create 1 augmented image per original
)

augment_minority_classes(
    "training/images", 
    "training/labels",
    augmented_images_dir,
    augmented_labels_dir,
    class_id=7,
    augmentation_factor=8  # Convert to integer - will create 1 augmented image per original
)



print("Augmentation complete. Augmented images and labels saved to:")
print(f"- {augmented_images_dir}")
print(f"- {augmented_labels_dir}")

Augmentation complete. Augmented images and labels saved to:
- training/augmented_images
- training/augmented_labels


In [40]:
# remove tags after fully developed
# for file in os.listdir(augmented_images_dir):
#     src = os.path.join(augmented_images_dir, file)
#     move(src, "Dataset/train/images")
    
# for file in os.listdir(augmented_labels_dir):
#     src = os.path.join(augmented_labels_dir, file)
#     move(src, "Dataset/train/labels")

In [43]:
t='training/images'
for file in os.listdir(t):
    src = os.path.join(t, file)
    copy(src, augmented_images_dir)
l='training/labels'
for file in os.listdir(l):
    src = os.path.join(l, file)
    copy(src, augmented_labels_dir)

In [2]:
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()

results = model.train(data='split.yaml', epochs=5, batch=8, imgsz=(1242, 375))

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics 8.3.91  Python-3.8.20 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=split.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=(1242, 375), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning C:\Users\user\KITTI_detection\training\labels.cache... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\user\KITTI_detection\val\labels.cache... 1497 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1497/1497 [00:00<?, ?it/s]


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


...

                   all       1497       8231      0.703      0.673      0.718      0.469

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      3.89G     0.9784     0.7934      1.036         85       1248: 100%|██████████| 748/748 [06:06<00:00,  2.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:15<00:00,  6.25it/s]


                   all       1497       8231      0.745      0.733      0.769       0.52

5 epochs completed in 2.053 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.3MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics 8.3.91  Python-3.8.20 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:16<00:00,  5.71it/s]


                   all       1497       8231       0.75      0.732      0.769       0.52
                   Car       1338       5889      0.884      0.896      0.945      0.739
                   Van        427        570      0.844      0.779      0.883      0.672
                 Truck        220        227       0.83      0.846      0.896      0.701
            Pedestrian        359        916      0.761       0.59      0.696       0.35
        Person_sitting         20         39      0.566      0.538      0.472      0.268
               Cyclist        222        308      0.724      0.725      0.755      0.447
                  Tram         62         95      0.674       0.85      0.839      0.525
                  Misc        156        187       0.72      0.632      0.669      0.459
Speed: 0.7ms preprocess, 3.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train6
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=

In [3]:
os.makedirs('Models', exist_ok=True)
model.save('Models/model2.pt')

In [5]:
# mlflow ui --backend-store-uri "file:///C:/Users/user/KITTI_detection/mlruns"
import mlflow

# Use local MLflow tracking (default path is ./mlruns)
mlflow.set_tracking_uri("file:///C:/Users/user/KITTI_detection/mlruns")
mlflow.set_experiment("Autonomous Vehicle")


<Experiment: artifact_location='file:///C:/Users/user/KITTI_detection/mlruns/901954944670996395', creation_time=1746668551367, experiment_id='901954944670996395', last_update_time=1746668551367, lifecycle_stage='active', name='Autonomous Vehicle', tags={}>

In [6]:
print(mlflow.get_tracking_uri())

file:///C:/Users/user/KITTI_detection/mlruns


In [9]:
def log_timeseries_metrics(metrics_root):
    for dirpath, _, filenames in os.walk(metrics_root):
        for fname in filenames:
            full_path = os.path.join(dirpath, fname)
            try:
                with open(full_path) as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) == 3:
                            _, value, step = parts
                            rel_path = os.path.relpath(full_path, metrics_root)
                            metric_name = rel_path.replace("\\", "/")
                            mlflow.log_metric(metric_name, float(value), step=int(step))
            except Exception as e:
                print(f"Skipping metric {full_path}: {e}")

def log_params(params_root):
    for fname in os.listdir(params_root):
        fpath = os.path.join(params_root, fname)
        try:
            with open(fpath) as f:
                value = f.read().strip()
                mlflow.log_param(fname, value)
        except Exception as e:
            print(f"Skipping param {fpath}: {e}")

def log_tags(tags_root):
    for fname in os.listdir(tags_root):
        fpath = os.path.join(tags_root, fname)
        try:
            with open(fpath) as f:
                value = f.read().strip()
                mlflow.set_tag(fname, value)
        except Exception as e:
            print(f"Skipping tag {fpath}: {e}")

def log_artifacts(artifacts_root):
    if os.path.exists(artifacts_root):
        mlflow.log_artifacts(artifacts_root)

# ---- MAIN ----
run_path = r"C:\Users\user\KITTI_detection\runs\mlflow\821983930449980226\2568967d7c4a4f4785a69e357d074c21"
mlflow.set_tracking_uri("file:///C:/Users/user/KITTI_detection/mlruns")
mlflow.set_experiment("Autonomous Vehicle")

with mlflow.start_run(run_name="KITTI"):
    print("Logging metrics...")
    log_timeseries_metrics(os.path.join(run_path, "metrics"))
    
    print("Logging parameters...")
    log_params(os.path.join(run_path, "params"))

    print("Logging tags...")
    log_tags(os.path.join(run_path, "tags"))

    print("Logging artifacts...")
    log_artifacts(os.path.join(run_path, "artifacts"))

print("✅ All data logged to MLflow!")


Logging metrics...
Logging parameters...
Logging tags...
Logging artifacts...
✅ All data logged to MLflow!


In [4]:
results = model.predict(
    source=r'C:\Users\user\KITTI_detection\testing\image_2',
    conf=0.25,
    save=True,
    project='results',      # Custom project name
    name='test_annotationss', # Custom folder name
    verbose=False
)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to results\test_annotationss


In [6]:
import os
import random
import cv2
import matplotlib.pyplot as plt

# Define the folder path using an absolute path
folder_path = r'C:\Users\user\KITTI_detection\results\test_annotationss'

# Get a list of all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a figure with 5 subplots
fig, ax = plt.subplots(ncols=5, figsize=(20, 20))

for i in range(5):
    # Choose a random file from the list
    random_file = random.choice(files)
    # Build the full image path
    path = os.path.join(folder_path, random_file)
    # Read the image using OpenCV
    img = cv2.imread(path)
    
    if img is not None:
        # Convert from BGR to RGB for matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax[i].imshow(img_rgb)
        ax[i].set_title(f'Image name: {random_file}')
    else:
        ax[i].text(0.5, 0.5, 'Image not found',
                   horizontalalignment='center',
                   verticalalignment='center', fontsize=12)
        ax[i].set_title(f'Image name: {random_file}')

# Display the figure with all images
plt.show()


<Figure size 2000x2000 with 5 Axes>